# EDA de **Telefonía Fija: Penetración por Provincia**

Cuaderno para **Visual Studio/VS Code** (Python) con pasos documentados en Markdown y exportación del CSV 'limpio'.

**Columnas esperadas:** `Registro_ID, Provincia_ID, Año, Trimestre, Provincias, Accesos telefonía fija cada 100 hab, Accesos telefonía fija cada 100 hogares, TiempoTrim_ID`.

## 1. Importaciones y configuración

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'axes.titlesize': 14,
    'axes.labelsize': 12
})

## 2. Carga del dataset desde la ruta indicada
> **Ruta proporcionada:**

`C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Telefonia Fija\\Telefonía Fija Penetración Provincias_con_ID.csv`

Usa `encoding='utf-8-sig'` para preservar acentos/eñes en Windows.

In [ ]:
ruta = r'C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Telefonia Fija\\Telefonía Fija Penetración Provincias_con_ID.csv'

df = pd.read_csv(ruta, encoding='utf-8-sig')
df_raw = df.copy()

print('Shape:', df.shape)
print('Columnas:', list(df.columns))
display(df.head())

## 3. Perfil inicial
- Tipos de datos (`df.info()`)
- Nulos y duplicados
- Estadísticos de penetración por 100 hab y por 100 hogares


In [ ]:
df.info()

print('
Nulos por columna:
', df.isna().sum())
print('Duplicados (filas completas):', df.duplicated().sum())

cols = ['Accesos telefonía fija cada 100 hab','Accesos telefonía fija cada 100 hogares']
for c in cols:
    if c in df.columns:
        print(c, 'min=', df[c].min(), 'max=', df[c].max(), 'mean=', float(df[c].mean()))

## 4. Validaciones de consistencia y construcción de `Periodo`
- Conversión a numérico en columnas clave
- Chequeo de valores negativos
- Construcción de `Periodo` (YYYY-Qn)


In [ ]:
num_cols = ['Registro_ID','Provincia_ID','Año','Trimestre','Accesos telefonía fija cada 100 hab','Accesos telefonía fija cada 100 hogares','TiempoTrim_ID']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

print('
Negativos por columna:
', (df[num_cols] < 0).sum())

df['Periodo'] = pd.PeriodIndex.from_fields(year=df['Año'], quarter=df['Trimestre']).astype(str)

## 5. Limpieza y ordenamiento
- Eliminación de duplicados
- Orden por `Año`, `Trimestre`, `Provincia_ID`


In [ ]:
df = df.drop_duplicates().sort_values(['Año','Trimestre','Provincia_ID']).reset_index(drop=True)
display(df.head())

## 6. Variables derivadas por provincia
- `PenHab_QoQ_%` y `PenHab_YoY_%` (por 100 hab)
- `PenHog_QoQ_%` y `PenHog_YoY_%` (por 100 hogares)

In [ ]:
df = df.sort_values(['Provincia_ID','Año','Trimestre']).reset_index(drop=True)
df['PenHab_QoQ_%'] = df.groupby('Provincia_ID')['Accesos telefonía fija cada 100 hab'].pct_change() * 100
df['PenHab_YoY_%'] = df.groupby('Provincia_ID')['Accesos telefonía fija cada 100 hab'].pct_change(4) * 100
df['PenHog_QoQ_%'] = df.groupby('Provincia_ID')['Accesos telefonía fija cada 100 hogares'].pct_change() * 100
df['PenHog_YoY_%'] = df.groupby('Provincia_ID')['Accesos telefonía fija cada 100 hogares'].pct_change(4) * 100

display(df[['Año','Trimestre','Provincias','Accesos telefonía fija cada 100 hab','PenHab_QoQ_%','PenHab_YoY_%']].tail(8))

## 7. Visualizaciones
- **Top 10** provincias por `Accesos cada 100 hab` (último trimestre)
- Líneas de penetración para provincias seleccionadas
- Heatmap de penetración por 100 hogares (provincias x periodos)

In [ ]:
# 7.1 Top 10 por 100 hab en el último trimestre
ultimo = df[['Año','Trimestre']].max().to_dict()
filtro_ult = (df['Año'] == ultimo['Año']) & (df['Trimestre'] == ultimo['Trimestre'])
top = df.loc[filtro_ult, ['Provincias','Accesos telefonía fija cada 100 hab']].sort_values('Accesos telefonía fija cada 100 hab', ascending=False).head(10)
plt.figure()
sns.barplot(data=top, x='Provincias', y='Accesos telefonía fija cada 100 hab', color='#2E86C1')
plt.xticks(rotation=45, ha='right')
plt.title('Top 10 provincias por accesos cada 100 hab (último trimestre)')
plt.xlabel('Provincia'); plt.ylabel('Accesos/100 hab'); plt.tight_layout(); plt.show()

# 7.2 Serie temporal para provincias clave
sel = ['Buenos Aires','CABA','Córdoba','Santa Fe']
df_sel = df[df['Provincias'].isin(sel)]
plt.figure()
sns.lineplot(data=df_sel, x='Periodo', y='Accesos telefonía fija cada 100 hab', hue='Provincias', marker='o')
plt.xticks(rotation=45, ha='right')
plt.title('Penetración (accesos/100 hab) por provincia seleccionada')
plt.xlabel('Periodo'); plt.ylabel('Accesos/100 hab'); plt.legend(title='Provincia'); plt.tight_layout(); plt.show()

# 7.3 Heatmap por 100 hogares
pivot = df.pivot_table(index='Provincias', columns='Periodo', values='Accesos telefonía fija cada 100 hogares')
plt.figure(figsize=(14, 10))
sns.heatmap(pivot, cmap='viridis', linewidths=0.5)
plt.title('Penetración telefonía fija por 100 hogares (heatmap)')
plt.xlabel('Periodo'); plt.ylabel('Provincia'); plt.tight_layout(); plt.show()

## 8. Guardado del archivo **limpio**
Se guarda el CSV en la **misma ruta** con la **leyenda `_limpio`** agregada al nombre, codificación `utf-8-sig`.

In [ ]:
ruta_path = Path(ruta)
ruta_limpia = ruta_path.with_name(ruta_path.stem + '_limpio' + ruta_path.suffix)
df.to_csv(ruta_limpia, index=False, encoding='utf-8-sig')
print('Guardado en:', ruta_limpia)